# Dependencies

In [2]:
# importing the necessary packages
import requests
from bs4 import BeautifulSoup
from datetime import date

today = date.today()
print('Today is ', today)

Today is  2021-10-20


# HTML scraper

In [3]:
###Scrape Market Watch
def scrape_ticker_mw(ticker):
    
    url = 'https://www.marketwatch.com/investing/stock/'+ticker.lower()+'?mod=over_search'
    r1 = requests.get(url)
    coverpage = r1.content
    soup1 = BeautifulSoup(coverpage, 'html.parser')#'html5lib')
    coverpage_news = soup1.find_all('div', class_="article__content")
    
    articles = []
    for article in coverpage_news:
        try:
            title = article.find('h3', class_ = 'article__headline').get_text()
            date = (article.find('span', class_ = 'article__timestamp'))['data-est']
            author = (article.find('span', class_ ='article__author')).get_text().strip('by ')
            articles.append([date.strip(), author.strip(), title.strip()])

        except:
            pass

    for article in articles:
        print(article)
        print('\n')

        
    return articles



In [4]:
###Scrape Google Finance
def scrape_ticker_gf(ticker):
    
    url = 'https://www.google.com/finance/quote/'+ticker.upper()+':NASDAQ'
    r1 = requests.get(url)
    coverpage = r1.content
    soup1 = BeautifulSoup(coverpage, 'html.parser')#'html5lib')
    coverpage_news = soup1.find_all('div', class_="nkXTJ")
    
    articles = []
    for article in coverpage_news:
        #print(article)
        try:
            title = article.find('div', class_ = 'AoCdqe').get_text().replace('\n','')
            
            date = (article.find('div', class_ = 'Adak')).get_text()
            date = date.split()
            author = (article.find('div', class_ ='sfyJob')).get_text()
            if date[1].replace('s','') !='month' and date[1].replace('s','') !='week':
                
                if date[1].replace('s','') !='day':
                    date = today - - timedelta(days=date[0])
                    articles.append([date, author, title])
                    
                elif date[1].replace('s','') !='hour':
                    date = today - - timedelta(hours=date[0])
                    articles.append([date, author, title])

                else:
                    print('Strange date')
                    print(date)
            else:
                print('Ignoring this')
                print(date)

        except:
            pass

    for article in articles:
        print(article)
        print('\n')
    return articles



In [5]:
###Scrape FinViz
def scrape_ticker_fv(ticker, verbose = 1, filtering = 0):

    headers = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15'}
    url = 'https://www.finviz.com/quote.ashx?t='+ticker
    r1 = requests.get(url, headers=headers)
    coverpage = r1.content
    soup1 = BeautifulSoup(coverpage, 'html.parser')#'html5lib')
    table = soup1.find('table', id='news-table', class_ = 'fullview-news-outer')    #table = soup1.findAll('table')
    rows = table.findAll('tr')
    
    dict_months = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05', 'Jun':'06', 'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}

    day_ = 'none'
    articles = []
    for article in rows:
        #print(article)
        try:
            title = article.find('a', class_ = 'tab-link-news').get_text()
            if filtering != 0 and filtering not in title.lower():
                continue
                
            date = article.find('td').get_text().strip().split()

            if len(date)==1:
                date = " ".join([day_, date[0]])
            elif len(date) == 2:
                day = date[0].split('-')
                day_ = "-".join(['20'+day[2],dict_months[day[0]], day[1]])
                date = " ".join([day_, date[1]])
            else:
                print('Date is strange')
                print(date)
                print(1/0)
                
            author = article.find('span').get_text()                
            articles.append([date, author, title.strip()])
        except:
            pass
        

    if verbose == 1:
        for article in articles:
            print(article)
            print('\n')
    return articles



# Examples

In [6]:
if __name__ == "__main__":
    articles = scrape_ticker_fv('aapl', verbose = 0, filtering = 'apple')
    for a in articles[:5]: print(a, '\n')

['2021-10-19 10:42PM', ' American City Business Journals', "The #AppleToo organizer Apple just fired didn't expect to be terminated  but isn't sorry about speaking out"] 

['2021-10-19 03:06PM', " Investor's Business Daily", 'Dow Jones Gains As Apple Pops; Biden Makes Spending Move; Bitcoin ETF Jumps On Debut'] 

['2021-10-19 12:32PM', " Investor's Business Daily Video", 'Apple Reclaims Key Level'] 

['2021-10-19 12:20PM', ' Yahoo Finance Video', 'Apple debuts new MacBook Pros with M1 Pro and M1 Max chips'] 

['2021-10-19 11:39AM', ' Investopedia', 'Apple (AAPL) Is Early Phone Leader in Heated 5G Race'] 

